In [43]:
''' Imports '''
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.integrate import trapezoid

In [37]:
'''Global var'''
TO_FEW = 5
LOWEST_VAR = 1e-5
DATA_SOURCE = '../data/source.csv'

In [38]:
'''Loading and displayng few records'''
source_file = pd.read_csv(DATA_SOURCE, index_col=0)
source_file.head()

,Serial,Source,X1,X2,X3,X4,X5,X6,X7,-01,...,ong,ong-,ria,rial,ser,seri,seria,tol,tolo,tolon
index,,,,,,,,,,,,,,,,,,,,,
0,MNOJUE-084279,isOK,6,6,6,0,13,6,0,0,...,0,0,0,0,0,0,0,0,0,0
1,JKLPDY12-13,isOK,6,4,4,0,11,6,0,0,...,0,0,0,0,0,0,0,0,0,0
2,WDS49kAUY,isNotOK,7,2,2,0,9,6,1,0,...,0,0,0,0,0,0,0,0,0,0
3,MNORSP-063772,isOK,6,6,6,0,13,6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MNOMPN-021264,isOK,6,6,6,0,13,6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
''' Binary lable selecting only features columns'''

source_file["label"] = source_file["Source"].map({"isOK": 0, "isNotOK": 1})

feature_cols = source_file.select_dtypes(include=[np.number]).columns.drop(["index", "label"], errors='ignore')

In [40]:
'''Drop low var columns'''
low_var_cols = source_file[feature_cols].var() < LOWEST_VAR
feature_cols = feature_cols[~low_var_cols]

In [44]:
'''Deleta calculation'''
delta_scores = {}

for feature in tqdm(feature_cols, desc="Obliczanie delta_j"):
    
    ok_vals = source_file[source_file["label"] == 0][feature].dropna()
    err_vals = source_file[source_file["label"] == 1][feature].dropna()
    
    # dont calculet rare features
    if len(ok_vals) < TO_FEW or len(err_vals) < TO_FEW:
        continue

    # probalibity
    try:
        kde_ok = gaussian_kde(ok_vals)
        kde_err = gaussian_kde(err_vals)
    except np.linalg.LinAlgError:
        continue 

    x_min = min(ok_vals.min(), err_vals.min())
    x_max = max(ok_vals.max(), err_vals.max())
    x_vals = np.linspace(x_min, x_max, 100)

    diff_squared = (kde_ok(x_vals) - kde_err(x_vals)) ** 2

    delta_j = trapezoid(diff_squared, x_vals)
    delta_scores[feature] = delta_j

Obliczanie delta_j: 100%|██████████| 107/107 [00:03<00:00, 28.98it/s]


In [45]:
# Posortuj cechy od najbardziej istotnych
sorted_features = sorted(delta_scores.items(), key=lambda x: x[1], reverse=True)

# Zamień na DataFrame (opcjonalnie)
import pandas as pd
delta_df = pd.DataFrame(sorted_features, columns=['feature', 'delta_j'])

# Wyświetl top 10
print(delta_df.head(10))

  feature     delta_j
0     eri  719.254018
1    long  357.898528
2     ria  237.777660
3     ng-  237.771791
4     -to  141.936010
5     olo  118.039586
6     ong   88.216565
7     lon   70.350146
8     mno   30.502739
9     -03   22.819662
